# Geração de Dados Fictícios - PNAB Ciclo 2 (Fase 1)

## Objetivo
Este notebook gera DataFrames fictícios usando Python e Pandas para simular os dados da **Fase 1 (Inscrição)** do Padrão SNIIC - PNAB Ciclo 2.

O objetivo é criar datasets realistas para testes de importação, validação de sistemas e desenvolvimento de módulos de exportação (Cenários 2 e 4 do Guia de Implementação).

Ele cria 7 DataFrames distintos, um para cada entidade da Fase 1:
1.  `Agente Individual`
2.  `Pessoa Jurídica`
3.  `Coletivo Informal`
4.  `Proposta de Ação Cultural`
5.  `Proposta de Subsídio`
6.  `Proposta de Bolsa`
7.  `Proposta de Premiação`

## Metodologia
O script utiliza as seguintes bibliotecas Python:
* `pandas` para a criação e estruturação dos DataFrames.
* `faker` para gerar dados realistas (nomes, CPFs, CNPJs, datas, etc.) em português brasileiro.
* `requests` para baixar o arquivo de Listas de Valores diretamente do repositório oficial no GitHub.
* `json` e `random` para ler o arquivo JSON e selecionar aleatoriamente os valores dos vocabulários controlados (ex: Segmento Cultural, Pautas Temáticas).

## Arquivos de Referência
Este notebook é baseado nos seguintes documentos oficiais do Padrão de Dados:

1.  **[Padrão SNIIC para Coleta de Dados - PNAB Ciclo 2](https://github.com/SGE-Subsecretaria-de-Gestao-Estrategica/aldir_blanc_ciclo2/blob/main/README.md)** O documento principal que descreve a política de coleta.

2.  **[Dicionários da Fase 1](https://github.com/SGE-Subsecretaria-de-Gestao-Estrategica/aldir_blanc_ciclo2/blob/main/dicionario-fase-1.md)** - Define os nomes das colunas (`Nomes formatados`) e os tipos de dados usados neste script.

3.  **[Listas de Valores (JSON)](https://raw.githubusercontent.com/SGE-Subsecretaria-de-Gestao-Estrategica/aldir_blanc_ciclo2/refs/heads/main/arquivos_e_modelos/lista_de_valores.json)** O arquivo JSON que este script **lê diretamente** para popular os campos de vocabulário controlado.

In [1]:
import pandas as pd
import random
import json
!pip install faker
import requests
from faker import Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.2 MB/s eta 0:00:00


In [2]:
# Inicializa o Faker para gerar dados em português brasileiro
fake = Faker('pt_BR')

In [3]:
# --- Carrega as Listas de Valores do URL do GitHub ---
url_json = "https://raw.githubusercontent.com/SGE-Subsecretaria-de-Gestao-Estrategica/aldir_blanc_ciclo2/refs/heads/main/arquivos_e_modelos/lista_de_valores.json"

try:
    response = requests.get(url_json)
    response.raise_for_status()  # Verifica se houve erro no download
    listas_json = response.json()
    print("Arquivo JSON carregado com sucesso do GitHub.")
except requests.exceptions.RequestException as e:
    print(f"ERRO: Não foi possível carregar o arquivo JSON do URL: {e}")
    exit()
except json.JSONDecodeError:
    print("ERRO: O arquivo baixado do URL não é um JSON válido.")
    exit()

Arquivo JSON carregado com sucesso do GitHub.


In [4]:
listas_json

{'1. Povos e comunidades tradicionais': ['Não pertence a povos ou comunidades tradicionais.',
  'Andirobeiros',
  'Apanhadores de flores sempre vivas',
  'Benzedeiros',
  'Caatingueiros',
  'Caboclos',
  'Caiçaras',
  'Catadores de mangaba',
  'Cipozeiros',
  'Comunidades de fundos e fechos de pasto',
  'Comunidades quilombolas',
  'Extrativistas',
  'Extrativistas costeiros e marinhos',
  'Faxinalenses',
  'Geraizeiros',
  'Ilhéus',
  'Juventude de povos e comunidades tradicionais',
  'Morroquianos',
  'Pantaneiros',
  'Pescadores artesanais',
  'Povo pomerano',
  'Povos ciganos',
  'Povos e comunidades de terreiro/de matriz africana',
  'Povos indígenas',
  'Quebradeiras de coco babaçu',
  'Raizeiros',
  'Retireiros do Araguaia',
  'Ribeirinhos',
  'Vazanteiros',
  'Veredeiros'],
 '2. Gênero': ['Homem cisgênero',
  'Mulher cisgênero',
  'Homem trans',
  'Mulher trans',
  'Travesti',
  'Não binário',
  'Outra',
  'Prefere não responder'],
 '3. Orientação sexual': ['Lésbica',
  'Gay',


In [5]:
# --- Listas de Valores (Valores Curtos e Específicos, não presentes no JSON) ---
sim_nao_sei = ['Sim', 'Não', 'Não sei']
modalidade_reserva = [
    'Proposta selecionada em ampla concorrência', 'Reserva de vagas para pessoas negras',
    'Reserva de vagas para pessoas indígenas', 'Reserva de vagas para pessoas com deficiência',
    'Reserva de vagas para outros grupos'
]
formato_acao = [
    'Presencialmente em local fixo', 'Presencialmente itinerante', 'Remotamente/Online',
    'Em formato híbrido', 'Outros', 'Não aplicável'
]
porte_espaco = [
    'Pequeno: até 499 m²', 'Média: de 500 m² a 1999 m²',
    'Grande: acima de 2000 m²', 'Não se aplica'
]
categoria_atividades_bolsa = [
    'Pesquisa/Formação em educação patrimonial', 'Pesquisa/Formação artístico-cultural ou técnica',
    'Pesquisa/Formação em política e gestão cultural', 'Difusão/circulação artístico-cultural',
    'Participação em eventos', 'Intercâmbios ou residências artísticas', 'Outros (especificar)'
]
ambito_bolsa = ['Nacional', 'Internacional', 'Ambos (nacional e internacional)']


In [6]:

# --- Funções de Geração de Dados (Usando o JSON carregado) ---

def generate_agente_individual(num_rows):
    data = []
    for _ in range(num_rows):
        tipo_agente = random.choice(['Pessoa física', 'Microempreendedor individual - MEI'])
        record = {
            'tipo_agente_individual': tipo_agente,
            'cpf': fake.cpf(),
            'cnpj_mei': fake.cnpj() if tipo_agente == 'Microempreendedor individual - MEI' else None,
            'nome_completo': fake.name(),
            'nome_social_artistico': random.choice([fake.name(), None]),
            'data_nascimento': fake.date_of_birth(minimum_age=18, maximum_age=90).strftime('%d/%m/%Y'),
            'email': fake.email(),
            'telefone': fake.phone_number(),
            'cep': fake.postcode(),
            'uf_residencia': fake.state_abbr(),
            'municipio_residencia': fake.city(),
            'endereco_completo': fake.address(),
            'comunidade_tradicional': ", ".join(random.sample(listas_json["1. Povos e comunidades tradicionais"], k=random.randint(1, 2))),
            'mestre_cultura_popular': random.choice([True, False]),
            'genero': random.choice(listas_json["2. Gênero"]),
            'orientacao_sexual': random.choice(listas_json["3. Orientação sexual"]),
            'cor_raca': random.choice(listas_json["4. Cor/Raça/Etnia"]),
            'pessoa_com_deficiencia': ", ".join(random.sample(listas_json["5. Pessoa com deficiência"], k=random.randint(1, 2))),
            'escolaridade': random.choice(listas_json["6. Escolaridade"]),
            'faixa_renda': random.choice(listas_json["7. Renda média individual (R$)"]),
            'anos_experiencia_cultural': random.randint(0, 50),
            'acessou_recursos_5anos': random.choice(sim_nao_sei)
        }
        data.append(record)
    return pd.DataFrame(data)

def generate_pessoa_juridica(num_rows):
    data = []
    for _ in range(num_rows):
        record = {
            'tipo_pessoa_juridica': random.choice(['Pessoa Jurídica com fins lucrativos (empresas)', 'Pessoa Jurídica sem fins lucrativos (OSCs)']),
            'cnpj': fake.cnpj(),
            'razao_social': fake.company(),
            'nome_fantasia': random.choice([fake.company(), None]),
            'data_fundacao': fake.date_of_birth(minimum_age=1, maximum_age=100).strftime('%d/%m/%Y'),
            'nome_representante_legal': fake.name(),
            'cpf_representante_legal': fake.cpf(),
            'email_organizacao': fake.email(),
            'telefone_organizacao': fake.phone_number(),
            'cep_sede': fake.postcode(),
            'endereco_sede': fake.address(),
            'municipio_sede': fake.city(),
            'uf_sede': fake.state_abbr(),
            'anos_atuacao_cultural': random.randint(1, 100),
            'acessou_recursos_5anos_org': random.choice(sim_nao_sei)
        }
        data.append(record)
    return pd.DataFrame(data)

def generate_coletivo_informal(num_rows):
    data = []
    for _ in range(num_rows):
        record = {
            'nome_grupo_coletivo': fake.company(),
            'qtd_membros_coletivo': random.randint(2, 50),
            'nome_representante_coletivo': fake.name(),
            'cpf_representante_coletivo': fake.cpf(),
            'email_coletivo': fake.email(),
            'telefone_coletivo': fake.phone_number(),
            'cep_coletivo': fake.postcode(),
            'endereco_coletivo': fake.address(),
            'municipio_coletivo': fake.city(),
            'uf_coletivo': fake.state_abbr(),
            'anos_atuacao_coletivo': random.randint(1, 50),
            'acessou_recursos_5anos_col': random.choice(sim_nao_sei)
        }
        data.append(record)
    return pd.DataFrame(data)

def generate_proposta_acao_cultural(num_rows):
    data = []
    for _ in range(num_rows):
        record = {
            'id_edital': f"Edital-2025-{random.randint(100, 999)}",
            'selecionado_cotas': random.choice(modalidade_reserva),
            'titulo_proposta': fake.catch_phrase(),
            'resumo_proposta': fake.paragraph(nb_sentences=5),
            'valor_proposta': round(random.uniform(5000, 100000), 2),
            'formato_proposta': random.choice(formato_acao),
            'cep_local_realizacao': fake.postcode(),
            'qtd_pessoas_remuneradas': random.randint(1, 20),
            'segmento_cultural_principal': random.choice(listas_json["8. Segmento Cultural"]),
            'etapa_ciclo_cultural_principal': random.choice(listas_json["9. Etapas do Ciclo Cultural"]),
            'pauta_tematica_principal': random.choice(listas_json["10. Pautas Temáticas"]),
            'atua_territorio_prioritario': ", ".join(random.sample(listas_json["11. Territórios Prioritários"], k=random.randint(1, 3))),
            'entregas_previstas': ", ".join(random.sample(listas_json["12. Tipos de Entregas"], k=random.randint(1, 3)))
        }
        data.append(record)
    return pd.DataFrame(data)

def generate_proposta_subsidio(num_rows):
    data = []
    for _ in range(num_rows):
        record = {
            'id_edital': f"Edital-Subsidio-2025-{random.randint(100, 999)}",
            'selecionado_cotas_espaco': random.choice(modalidade_reserva),
            'nome_espaco_cultural': fake.company(),
            'cep_espaco': fake.postcode(),
            'tipo_espaco_cultural': ", ".join(random.sample(listas_json["13. Tipos de Espaços e Equipamentos Culturais"], k=random.randint(1, 3))),
            'porte_espaco_cultural': random.choice(porte_espaco),
            'segmentos_espaco_cultural': ", ".join(random.sample(listas_json["8. Segmento Cultural"], k=random.randint(1, 3))),
            'espaco_territorio_prioritario': ", ".join(random.sample(listas_json["11. Territórios Prioritários"], k=random.randint(1, 2))),
            'anos_existencia_espaco': random.randint(1, 70),
            'acessou_recursos_5anos_esp': random.choice(sim_nao_sei),
            'valor_total_subsidio': round(random.uniform(10000, 50000), 2),
            'qtd_parcelas_subsidio': random.randint(1, 12)
        }
        data.append(record)
    return pd.DataFrame(data)

def generate_proposta_bolsa(num_rows):
    data = []
    for _ in range(num_rows):
        gera_entrega = random.choice([True, False])
        ambito = random.choice(ambito_bolsa)
        record = {
            'id_edital': f"Edital-Bolsa-2025-{random.randint(100, 999)}",
            'selecionado_cotas': random.choice(modalidade_reserva),
            'titulo_proposta': fake.catch_phrase(),
            'resumo_proposta': fake.paragraph(nb_sentences=4),
            'valor_proposta': round(random.uniform(5000, 30000), 2),
            'qtd_pessoas_remuneradas': random.randint(1, 3),
            'segmento_cultural_principal': random.choice(listas_json["8. Segmento Cultural"]),
            'pauta_tematica_principal': random.choice(listas_json["10. Pautas Temáticas"]),
            'categoria_atividades_bolsa': random.choice(categoria_atividades_bolsa),
            'ambito_bolsa': ambito,
            'destino_nacional_bolsa': fake.city() if ambito in ['Nacional', 'Ambos (nacional e internacional)'] else None,
            'destino_internacional_bolsa': fake.country() if ambito in ['Internacional', 'Ambos (nacional e internacional)'] else None,
            'gera_produto_entrega': gera_entrega,
            'tipo_entrega_prevista': random.choice(listas_json["12. Tipos de Entregas"]) if gera_entrega else None
        }
        data.append(record)
    return pd.DataFrame(data)

def generate_proposta_premiacao(num_rows):
    data = []
    for _ in range(num_rows):
        record = {
            'id_edital': f"Edital-Premio-2025-{random.randint(100, 999)}",
            'selecionado_cotas': random.choice(modalidade_reserva),
            'valor_premiacao': round(random.uniform(1000, 20000), 2),
            'segmento_cultural_principal': random.choice(listas_json["8. Segmento Cultural"]),
            'pauta_tematica_principal': random.choice(listas_json["10. Pautas Temáticas"]),
            'atua_territorio_prioritario': ", ".join(random.sample(listas_json["11. Territórios Prioritários"], k=random.randint(1, 2)))
        }
        data.append(record)
    return pd.DataFrame(data)

In [7]:
NUM_ROWS = 50  # Define quantos registros fictícios gerar por entidade

# 1. Gerar todos os DataFrames
df_agente_individual = generate_agente_individual(NUM_ROWS)
df_pessoa_juridica = generate_pessoa_juridica(NUM_ROWS)
df_coletivo_informal = generate_coletivo_informal(NUM_ROWS)
df_proposta_acao = generate_proposta_acao_cultural(NUM_ROWS)
df_proposta_subsidio = generate_proposta_subsidio(NUM_ROWS)
df_proposta_bolsa = generate_proposta_bolsa(NUM_ROWS)
df_proposta_premiacao = generate_proposta_premiacao(NUM_ROWS)

# 2. Salvar todos em CSV
# Usar encoding 'utf-8-sig' garante compatibilidade com Excel (acentos)
df_agente_individual.to_csv("fase1_agente_individual.csv", index=False, encoding='utf-8-sig')
df_pessoa_juridica.to_csv("fase1_pessoa_juridica.csv", index=False, encoding='utf-8-sig')
df_coletivo_informal.to_csv("fase1_coletivo_informal.csv", index=False, encoding='utf-8-sig')
df_proposta_acao.to_csv("fase1_proposta_acao_cultural.csv", index=False, encoding='utf-8-sig')
df_proposta_subsidio.to_csv("fase1_proposta_subsidio.csv", index=False, encoding='utf-8-sig')
df_proposta_bolsa.to_csv("fase1_proposta_bolsa.csv", index=False, encoding='utf-8-sig')
df_proposta_premiacao.to_csv("fase1_proposta_premiacao.csv", index=False, encoding='utf-8-sig')

print("Todos os 7 arquivos CSV da Fase 1 foram gerados com sucesso.")

Todos os 7 arquivos CSV da Fase 1 foram gerados com sucesso.


In [8]:
df_agente_individual

,tipo_agente_individual,cpf,cnpj_mei,nome_completo,nome_social_artistico,data_nascimento,email,telefone,cep,uf_residencia,...,comunidade_tradicional,mestre_cultura_popular,genero,orientacao_sexual,cor_raca,pessoa_com_deficiencia,escolaridade,faixa_renda,anos_experiencia_cultural,acessou_recursos_5anos
0,Pessoa física,824.953.160-42,None,Maria Liz da Rosa,Juliana Brito,04/11/1964,julia55@example.net,+55 (084) 7812 8248,20681470,RJ,...,Ribeirinhos,True,Travesti,Heterossexual,Branca,"Sim, Múltipla, Sim, Intelectual",Ensino superior incompleto ou equivalente,Acima de 10 salários mínimos,5,Sim
1,Microempreendedor individual - MEI,209.674.135-16,63.809.572/0001-36,Julia Novaes,Luiz Henrique Vargas,03/08/2001,vianavinicius@example.net,+55 (011) 6581 1600,43859-779,RN,...,"Ilhéus, Retireiros do Araguaia",True,Mulher trans,Bissexual,Amarela,"Sim, Visual, Sim, Múltipla",Ensino fundamental completo ou equivalente,De 5 a 8 salários mínimos,19,Sim
2,Microempreendedor individual - MEI,087.963.524-00,25.876.093/0001-43,Bella Fogaça,Cauã Oliveira,01/09/1942,novaesmaria-vitoria@example.net,(031) 4184-8232,47619008,MG,...,"Veredeiros, Caiçaras",False,Prefere não responder,Prefere não responder,Preta,"Sim, Múltipla, Sim, outra (especificar)",Ensino médio completo ou equivalente,De 8 a 10 salários mínimos,38,Não sei
3,Microempreendedor individual - MEI,324.689.571-64,56.984.103/0001-98,Luigi Sampaio,None,25/07/1995,pedro-miguelvieira@example.com,(071) 6865-6527,52206-306,RJ,...,Povos indígenas,True,Não binário,Heterossexual,Amarela,"Sim, Visual",Curso técnico incompleto,Nenhuma renda,23,Sim
4,Microempreendedor individual - MEI,836.250.719-59,26.801.374/0001-08,Sra. Rebeca Silva,Dra. Joana Nunes,14/03/2005,sousaana-carolina@example.org,11 0860 6899,64201717,AM,...,"Juventude de povos e comunidades tradicionais,...",True,Travesti,Lésbica,Preta,"Sim, outra (especificar), Sim, Transtorno do E...",Pós-graduação incompleto,Acima de 10 salários mínimos,20,Sim
5,Pessoa física,396.217.854-64,None,João Gabriel Azevedo,None,19/09/1969,egoncalves@example.net,+55 84 2870 1966,99305-997,MG,...,"Comunidades de fundos e fechos de pasto, Vered...",False,Homem trans,Outra,Preta,"Sim, Física-motora, Sim, Transtorno do Espectr...",Ensino superior incompleto ou equivalente,Acima de 10 salários mínimos,2,Não sei
6,Pessoa física,451.376.928-91,None,Danilo Marques,Sarah Farias,03/08/1978,lucas-gabriel07@example.org,(041) 9817-1786,81377975,AM,...,"Ribeirinhos, Povos e comunidades de terreiro/d...",True,Mulher cisgênero,Heterossexual,Branca,"Sim, outra (especificar)",Não possui educação formal,Até 1 salário mínimo,21,Não
7,Microempreendedor individual - MEI,674.219.058-20,87.610.293/0001-07,Maria Luiza Fogaça,Lunna Almeida,11/10/1975,kramos@example.org,+55 (011) 6114-9126,82152-318,ES,...,Pantaneiros,False,Não binário,Gay,Amarela,"Não, Sim, Transtorno do Espectro Autista",Ensino médio incompleto ou equivalente,De 5 a 8 salários mínimos,31,Não
8,Pessoa física,082.935.761-03,None,Dra. Maitê Garcia,None,07/06/1939,jlopes@example.net,+55 61 3664 0444,93212-581,MS,...,Povos e comunidades de terreiro/de matriz afri...,False,Homem trans,Lésbica,Parda,"Sim, Intelectual",Ensino superior completo,Até 1 salário mínimo,9,Sim
9,Microempreendedor individual - MEI,178.459.230-79,53.186.479/0001-69,Allana Rios,None,02/03/1953,novaesrafael@example.com,(051) 1818 7246,46833-943,RS,...,"Povo pomerano, Caboclos",True,Mulher trans,Gay,Branca,"Sim, Múltipla, Não",Ensino fundamental incompleto ou equivalente,Acima de 10 salários mínimos,46,Não
